In [1]:
import os, sys
parent_dir = os.path.realpath('..')
sys.path.append(parent_dir)  # parent directory path adding

In [ ]:
import pandas as pd
import numpy as np
import logging
import json

import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

from src.libs.dataset_utils import SinDataset
from src.data.trigonometric_dataset import *
from src.models.sin_LSTM import SinLSTM
from src.libs.train_valid_test_utils import TrainValidEvaluate
from src.libs.visualize_utils import VisualizeUtils

In [ ]:
import sys

print(sys.executable)

In [ ]:
device = 'cpu'
model_name = 'sin_lstm'
hidden_dim1 = 100
batch_size = 16
num_layers = 2
weight_decay = 1e-6
learning_rate = 1e-3
n_epochs = 50
sin_data_number = 1000
sin_data_interval = 20
data_sequence_length = 10
device = 'cpu'

# Generate Data

In [ ]:
save_name = f"{model_name}_hd1_{hidden_dim1}_nl_{num_layers}_lr_{learning_rate}"
save_path = f"./models/{model_name}/{save_name}.pth"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
total_data = generate_sin_data(
    d_num=sin_data_number,
    data_interval=sin_data_interval,
    data_sequence=data_sequence_length
)

In [ ]:
train_dataset = SinDataset(total_data=total_data, data_type='train')
valid_dataset = SinDataset(total_data=total_data, data_type='valid')
test_dataset = SinDataset(total_data=total_data, data_type='test')

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

# Modeling

In [ ]:
lstm_input_dim = len(train_dataset[0][0][-1])

In [ ]:
model = SinLSTM(input_dim=lstm_input_dim, lstm_hidden_dim=hidden_dim1, num_layers=num_layers).to(device)

# Training

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             weight_decay=weight_decay)

In [ ]:
criterion = nn.MSELoss()

In [ ]:
train_valid_evaluate = TrainValidEvaluate(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    n_epochs=n_epochs,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    path=save_path,
    logging=logging,
)

In [ ]:
train_valid_evaluate.sin_train()

In [ ]:
train_valid_evaluate.sin_test()

In [ ]:
print(f'TEST SCORE :: \n \ttest MAPE={train_valid_evaluate.avg_test_mape:.4f} \n\ttest loss={train_valid_evaluate.avg_test_loss:.4f}')

# Visualize

In [ ]:
visualize_util = VisualizeUtils(predict_value=train_valid_evaluate.test_prediction_result,
                                true_value=train_valid_evaluate.test_target_result,
                                data_interval=sin_data_interval)

In [ ]:
visualize_util.visualize_sin_data('sin prediction result')